In [1]:
###############################################
## Import necessary libraries and files
###############################################
import numpy as cp
try:
    import cupy
    if cupy.cuda.is_available():
        cp = cupy
except:
    pass
import matplotlib.pyplot as plt
import time
import sys
from class_rbm import class_RBM
from aux import binarize_data
from aux import get_dataset
from aux import load_acc

In [ ]:
###############################################
## Control Panel
###############################################

model = {
    'N':784,                  # number of visible units
    'M':100,                  # number of hidden units
    'eta':0.1,                # learning rate of the RBM
    'batch_size':100,         # batch size
    'W_init':'lecun',         #{'lecun', 'std'} --weight initialization
    'n_cl': 10                # number of classes (C)
}

training_settings = {
'dataset': 'mnist',            #{'mnist','kmnist'} --dataset
'KH': 'hKH  ',                 #{None, 'hKH', 'vKH', 'hvKH'} --KH Modulation approach
'training': 'CD',              #{'CD', 'PCD'} --training algorithm of the RBM
'k': 1,                        # number of Gibbs sampling steps
'epochs': 3,                   # number of epochs (S)
'eps0': 4e-2,                  # KH learning rate
'eps_d': True,                 #{False, True} -- decays with a polynomial eps0*(1- t/epochs)^(3/2)
'delta': 0.4,                  # KH penalty term
'p': 2.0,                      # KH Lebesgue power
'l': 2,                        # KH ranking parameter l >= 2
'R': 1.0 if model['W_init'] == 'std' else 0.1, # Lebesgue p norm radius
'seed': 1234,                  # random seed
'addrss': '', #'out/'
'label': 'p1.5decay'
}
###############################################
## Loading and preprocessing dataset
###############################################

train_data, valid_data, test_data, train_labels, valid_labels, test_labels = get_dataset(dataset=training_settings['dataset'])

try:
    if cp == cupy:
        train_data = cp.asarray(train_data)
        valid_data = cp.asarray(valid_data)
        test_data = cp.asarray(test_data)
        train_labels = cp.asarray(train_labels)
        valid_labels = cp.asarray(valid_labels)
        test_labels = cp.asarray(test_labels)
except:
    pass

train_data = cp.concatenate((train_data,valid_data))
train_labels = cp.concatenate((train_labels,valid_labels))

# data binarization
X_train = binarize_data(train_data)
X_test = binarize_data(test_data)


X_train = cp.concatenate((X_train,train_labels),axis=1)
X_test = cp.concatenate((X_test,test_labels),axis=1)

2024-09-10 18:34:03.917527: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 18:34:06.054368: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-09-10 18:34:06.186112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2024-09-10 18:34:06.186572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start abor

In [3]:
start_time = time.time()
cp.random.seed(seed=training_settings['seed'])
rbm = class_RBM(model)
rbm.train(X_train,X_test,training_settings,incr=1,save_checkpoints=False,track_learning=True,save_learn_funcs=True,
          save_params=True,plot_weights=True)
end_time = time.time()  # Record the end time for the entire training process
total_training_time = end_time - start_time
print('Total training time: %.2f seconds' % total_training_time)

NameError: name 'eta' is not defined

In [4]:
r = load_acc(model,training_settings)
r

array([0.8665, 0.8861, 0.8956])